In [1]:
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor
import pandas as pd
import os
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.svm import LinearSVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import fetch_openml

rng = np.random.default_rng(42)

In [ ]:
# 8. Train a LinearSVC on a linearly separable dataset. Then train an SVC and a SGDClassifier on the same dataset.
#    See if you can get them to produce roughly the same model

In [ ]:
# create a linearly separable dataset
# for this example, I will create a positive class (y = 1) centered around (1, 1)
# and a negative class (y = 0) centered around (-1, -1)
# each instance has random uniform error, on each axis, in the interval [-.5, .5)
# each class has 100 instances

In [ ]:
X_positive = rng.random((100, 2)) + .5
y_positive = np.full((100), 1)

X_negative = -1 * rng.random((100, 2)) - .5
y_negative = np.full((100), 0)

In [ ]:
# brief validation to show that there are no values in either column of X_positive that are negative
# and there are no values in either column of X_negative that are positive
# which guarantees that the dataset is linearly separable

In [ ]:
X_positive[(X_positive[:, 0] < 0) | (X_positive[:, 1] < 0)]

In [ ]:
X_negative[(X_negative[:, 0] > 0) | (X_negative[:, 1] > 0)]

In [ ]:
# brief validation to show that X_positive and X_negative are centered around (1, 1) and (-1, -1), respectively

In [ ]:
X_positive.mean(axis=0)

In [ ]:
X_negative.mean(axis=0)

In [ ]:
# glue the datasets together, then shuffle

In [ ]:
X = np.append(X_positive, X_negative, axis=0)
y = np.append(y_positive, y_negative, axis=0)

X, y = shuffle(X, y, random_state=42)

In [ ]:
# briefly verify that training instances and labels were shuffled correctly
# i.e. positive instances assigned to positive class and negative to negative

In [ ]:
((np.apply_along_axis(sum, 1, X) >= 0).astype(int) == y)[False]

In [ ]:
# normally I would create a test set and perform cross-validation but for this example I only care about
# the similarity of the trained models rather than their performance and generalizability.
# and besides, I know the exact logic of how the dataset was created because I created it,
# which is the ultimate form of data snooping

# so I will just train the models and see how similar I can make them

In [ ]:
# standard scale features

In [ ]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [ ]:
# train models

In [ ]:
linear_svc = LinearSVC(loss='hinge')
linear_svc.fit(X_scaled, y)
print(f'intercept : {linear_svc.intercept_} \nweights : {linear_svc.coef_}')

In [ ]:
svc = SVC(kernel='linear')
svc.fit(X_scaled, y)
print(f'intercept : {svc.intercept_} \nweights : {svc.coef_}')

In [ ]:
sgd_classifier = SGDClassifier(alpha=.005, tol=.0001, max_iter=1_000_000, n_iter_no_change=100_000, random_state=42)
sgd_classifier.fit(X_scaled, y)
print(f'intercept : {sgd_classifier.intercept_} \nweights : {sgd_classifier.coef_}')

In [ ]:
#################################################################################################################

In [2]:
# 9. Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers,
#    you will need to use one-versus-the-rest to classify all 10 digits. You may want to tune the
#    hyperparameters using small validation sets speed up the process. What accuracy can you reach?

# note that for LinearSVC there is a parameter for multi_class='ovr', but I will implement this myself for practice

In [3]:
# load data

In [4]:
mnist = fetch_openml('mnist_784', version=1)

features = mnist['data']

labels = mnist['target']
labels = labels.astype(np.uint16)

In [5]:
# create training and test set
# NOTE: data is preshuffled

In [6]:
train_features = features[:60000]
train_labels = labels[:60000]

test_features = features[60000:]
test_labels = labels[60000:]

In [7]:
train_features.shape

(60000, 784)

In [8]:
train_labels.shape

(60000,)

In [9]:
# create data preparer and prep data

In [10]:
mnist_scaler = StandardScaler()
prepped_train_features = mnist_scaler.fit_transform(train_features)
prepped_test_features = mnist_scaler.transform(test_features)

In [11]:
# create label matrix preparer

# takes label vector and turns it into a matrix where each column corresponds to a class (digit) and each row is an instance,
# where a 1 indicates the instance belongs to that class and a 0 indicates that it does not

In [12]:
class LabelMatrixPreparer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return
    
    def fit(self, X):
        self.classes_ = np.unique(X)
        return self
    
    def transform(self, X):
        label_matrix = None
        for class_ in self.classes_:
            new_column = (X == class_).astype(int)
            if label_matrix is None:
                label_matrix = new_column
            else:
                label_matrix = np.c_[label_matrix, new_column]
        return label_matrix

In [13]:
# quickly validate label matrix preparer

In [14]:
lmp = LabelMatrixPreparer()
train_label_matrix = lmp.fit_transform(train_labels)
validation_array = train_label_matrix[:, 4] == (train_labels == 4)
validation_array[validation_array == False]

Series([], Name: class, dtype: bool)

In [15]:
# create OVR estimator that trains one SVM for each class according to corresponding column in label matrix

In [35]:
class OVR_SVM_Estimator(BaseEstimator):
    def __init__(self, class_names, C=1):
        self.class_names = class_names
        self.C = C
        return
    
    def fit(self, X, y=None):
        self.estimators_ = []
        for col_idx in range(y.shape[1]):
            estimator = LinearSVC(C=self.C, max_iter=1_000_000)
            estimator.fit(X, y[:, col_idx])
            self.estimators_.append(estimator)
        return self
    
    def predict(self, X):
        decision_scores = None
        for estimator in self.estimators_:
            new_column = estimator.decision_function(X)
            if decision_scores is None:
                decision_scores = new_column
            else:
                decision_scores = np.c_[decision_scores, new_column]
        
        #self.class_names[arg_max]
        return decision_scores

In [ ]:
ovr = OVR_SVM_Estimator(list(range(10)))
ovr.fit(prepped_train_features, train_label_matrix)

In [ ]:
ovr.predict(prepped_train_features)[6]

In [ ]:
train_labels[6]

In [ ]:
# perform gridsearch on OVR estimator to select hyperparameter values

In [ ]:
# evaluate on test set

In [18]:
#################################################################################################################

In [2]:
# 10. Train an SVM regressor on the California housing dataset

In [3]:
# load data

In [4]:
raw_data = pd.read_csv('housing.csv')

In [5]:
raw_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
# add a column to raw_data that assigns which decile of median_house_value that instance belongs to

# note that this column is created to allow stratified sampling of the target in the creation of our test set.
# because the deciles are calculated on the entire dataset (train and test),
# it is important that we drop the column as soon as the train/test sets are formed.
# this column should NOT be treated as an input feature because, having been calculated on the full dataset,
# it is data snooping and will likely not generalize well to new data if used to train a model.
# the column exists purely to ensure a wide variety of targets exists in both the train and test sets

In [7]:
decile_1 = raw_data['median_house_value'].quantile(q=.1)
decile_2 = raw_data['median_house_value'].quantile(q=.2)
decile_3 = raw_data['median_house_value'].quantile(q=.3)
decile_4 = raw_data['median_house_value'].quantile(q=.4)
decile_5 = raw_data['median_house_value'].quantile(q=.5)
decile_6 = raw_data['median_house_value'].quantile(q=.6)
decile_7 = raw_data['median_house_value'].quantile(q=.7)
decile_8 = raw_data['median_house_value'].quantile(q=.8)
decile_9 = raw_data['median_house_value'].quantile(q=.9)

decile_bins = [-np.inf, decile_1, decile_2, decile_3, decile_4, decile_5, decile_6, decile_7, decile_8, decile_9, np.inf]

raw_data['decile'] = pd.cut(raw_data['median_house_value'], bins=decile_bins, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [8]:
# split into training and test sets, drop decile column, and separate into features and targets

In [9]:
stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=.2, random_state=42)
train_test_split = stratified_splitter.split(raw_data, raw_data['decile'])

train_set = None
test_set = None

for train_indices, test_indices in train_test_split:
    train_set = raw_data.loc[train_indices]
    test_set = raw_data.loc[test_indices]

train_targets = train_set['median_house_value'].copy()
test_targets = train_set['median_house_value'].copy()

train_features = train_set.drop(columns=['median_house_value', 'decile'])
test_features = test_set.drop(columns=['median_house_value', 'decile'])

In [10]:
# create variables for column names and identify indices for columns used in added numerical features

In [11]:
rooms_string = 'total_rooms'
bedrooms_string = 'total_bedrooms'
population_string = 'population'
households_string = 'households'
ocean_proximity_string = 'ocean_proximity'

train_features_column_names = list(train_features.columns)

rooms_index = train_features_column_names.index(rooms_string)
bedrooms_index = train_features_column_names.index(bedrooms_string)
population_index = train_features_column_names.index(population_string)
households_index = train_features_column_names.index(households_string)

bpr_string = 'bedrooms_per_room'
rph_string = 'rooms_per_household'
pph_string = 'population_per_household'

In [12]:
# create a transformer that adds the extra numerical features

In [13]:
class NumericalFeatureAdder(BaseEstimator, TransformerMixin):
    def __init__(self, numerical_features, add_bedrooms_per_room=False, add_rooms_per_household=False, add_population_per_household=False):
        self.numerical_features = numerical_features
        
        self.add_bedrooms_per_room = add_bedrooms_per_room
        self.add_rooms_per_household = add_rooms_per_household
        self.add_population_per_household = add_population_per_household
        return
    
    def fit(self, X):
        possible_features = (bpr_string, rph_string, pph_string)
        parameter_settings = (self.add_bedrooms_per_room, self.add_rooms_per_household, self.add_population_per_household)

        self.possible_features_and_parameter_settings = list(zip(possible_features, parameter_settings))
        
        self.added_features = []
        
        for possible_feature, parameter_setting in self.possible_features_and_parameter_settings:
            if parameter_setting:
                self.added_features.append(possible_feature)
        
        return self
    
    def transform(self, X):
        new_columns = None
        
        for possible_feature, parameter_setting in self.possible_features_and_parameter_settings:
            new_column = None
            if parameter_setting:
                if possible_feature == bpr_string:
                    new_column = X[:, bedrooms_index] / X[:, rooms_index]
                elif possible_feature == rph_string:
                    new_column = X[:, rooms_index] / X[:, households_index]
                elif possible_feature == pph_string:
                    new_column = X[:, population_index] / X[:, households_index]
                
                if new_columns is None:
                    new_columns = new_column
                else:
                    new_columns = np.c_[new_columns, new_column]
        
        if new_columns is None:
            return X
        
        return np.c_[X, new_columns]

In [14]:
# create a column transformer that performs data preparation

In [15]:
class DataPreparer(BaseEstimator, TransformerMixin):
    def __init__(self, numerical_features, categorical_features, add_bpr=False, add_rph=False, add_pph=False, polynomial_degree=1, interaction_terms_only=False):
        self.numerical_features = numerical_features        
        self.categorical_features = categorical_features
        
        self.add_bpr = add_bpr
        self.add_rph = add_rph
        self.add_pph = add_pph
        self.polynomial_degree = polynomial_degree
        self.interaction_terms_only = interaction_terms_only
        return
    
    def fit(self, X):
        numerical_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')),
                                           ('feature_adder', NumericalFeatureAdder(self.numerical_features, add_bedrooms_per_room=self.add_bpr, add_rooms_per_household=self.add_rph, add_population_per_household=self.add_pph)),
                                           ('polynomial_features', PolynomialFeatures(degree=self.polynomial_degree, interaction_only=self.interaction_terms_only, include_bias=False)),
                                           ('scaler', StandardScaler())])
        self.column_transformer = ColumnTransformer([('numerical', numerical_pipeline, self.numerical_features)
                                                       , ('categorical', OneHotEncoder(handle_unknown='ignore'), self.categorical_features)])
        
        self.column_transformer.fit(X)
        
        feature_adder = self.column_transformer.named_transformers_['numerical'].named_steps['feature_adder']
        self.added_numerical_features = feature_adder.added_features

        polynomial_features = self.column_transformer.named_transformers_['numerical'].named_steps['polynomial_features']
        numerical_features_plus_added_numerical_features = self.numerical_features + self.added_numerical_features
        self.all_numerical_features = [' * '.join([f'{numerical_features_plus_added_numerical_features[feature_index]}^{power_array[feature_index]}' for feature_index in range(len(power_array)) if power_array[feature_index] != 0]) for power_array in polynomial_features.powers_]
        # polynomial_features.get_feature_names_out(input_features=self.numerical_features + self.added_numerical_features)

        one_hot_encoder = self.column_transformer.named_transformers_['categorical']
        self.all_categorical_features = list(np.concatenate(one_hot_encoder.categories_))
        
        self.features = self.all_numerical_features + self.all_categorical_features
        return self
    
    def transform(self, X):
        return self.column_transformer.transform(X)

In [16]:
categorical_features = [ocean_proximity_string]
numerical_features = list(train_features.drop(columns=categorical_features))

data_preparer = DataPreparer(numerical_features, categorical_features, add_bpr=True, polynomial_degree=2, interaction_terms_only=True)
prepped_train_features = data_preparer.fit_transform(train_features)

data_preparer.features

['longitude^1',
 'latitude^1',
 'housing_median_age^1',
 'total_rooms^1',
 'total_bedrooms^1',
 'population^1',
 'households^1',
 'median_income^1',
 'bedrooms_per_room^1',
 'longitude^1 * latitude^1',
 'longitude^1 * housing_median_age^1',
 'longitude^1 * total_rooms^1',
 'longitude^1 * total_bedrooms^1',
 'longitude^1 * population^1',
 'longitude^1 * households^1',
 'longitude^1 * median_income^1',
 'longitude^1 * bedrooms_per_room^1',
 'latitude^1 * housing_median_age^1',
 'latitude^1 * total_rooms^1',
 'latitude^1 * total_bedrooms^1',
 'latitude^1 * population^1',
 'latitude^1 * households^1',
 'latitude^1 * median_income^1',
 'latitude^1 * bedrooms_per_room^1',
 'housing_median_age^1 * total_rooms^1',
 'housing_median_age^1 * total_bedrooms^1',
 'housing_median_age^1 * population^1',
 'housing_median_age^1 * households^1',
 'housing_median_age^1 * median_income^1',
 'housing_median_age^1 * bedrooms_per_room^1',
 'total_rooms^1 * total_bedrooms^1',
 'total_rooms^1 * population^1',


In [17]:
# brief validation of data preparer on first row of shuffled training features

In [18]:
train_features.shape

(16512, 9)

In [19]:
prepped_train_features.shape

(16512, 50)

In [20]:
train_features.iloc[0]

longitude             -116.5
latitude               33.82
housing_median_age      16.0
total_rooms            343.0
total_bedrooms          85.0
population              29.0
households              14.0
median_income         2.1042
ocean_proximity       INLAND
Name: 12344, dtype: object

In [21]:
prepped_train_features[0]

array([ 1.53652021, -0.8524502 , -1.00209463, -1.04711441, -1.0773835 ,
       -1.25320775, -1.26619991, -0.92674015,  0.54898116,  1.0060471 ,
        1.02989419,  1.05465074,  1.08641961,  1.25706504,  1.27015793,
        0.95387955, -0.45035876, -1.04951618, -1.06151519, -1.09593943,
       -1.26210364, -1.27462169, -0.98345352,  0.35578052, -1.41823592,
       -1.32731384, -1.44661167, -1.45043049, -0.99668143, -0.63188836,
       -0.34987324, -0.33080851, -0.3554938 , -0.79244735, -1.0773835 ,
       -0.36341573, -0.38006211, -0.9136977 , -0.94142066, -0.36437014,
       -0.97325909, -1.16058851, -0.99310491, -1.13858577, -0.91424779,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ])

In [22]:
(train_features.iloc[0, 0] - train_features['longitude'].mean())/train_features['longitude'].std()

1.5364736790267735

In [23]:
# create a data preparation and prediction pipeline to experiment with different hyperparameters in GridSearchCV

# I wanted to try implementing the SVR or LinearSVR class, but they both took far too long even on a broad grid search.
# so instead I am implementing SGDRegressor and using epsilon insensitive to capture the desired SVM behavior.

# furthermore, when fitting the estimator, I will calculate the standard deviation of the training target values
# and use this as a multiplier for the epsilon value. This will ensure the epsilon parameter is scaled appropriately
# to the magnitude of the target values

# https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html
# can probably use this to make code more concise
# all steps in a pipeline must be transformers, except for the final step. the final step can be a estimator

In [24]:
class DataPreparerAndPredictor(BaseEstimator):
    def __init__(self, numerical_features, categorical_features, add_bpr=False, add_rph=False, add_pph=False,
                 polynomial_degree=1, interaction_terms_only=False, epsilon=0, loss='epsilon_insensitive', alpha=0,
                 penalty='l2', l1_ratio=.15):
        self.numerical_features = numerical_features
        self.categorical_features = categorical_features
        
        self.add_bpr = add_bpr
        self.add_rph = add_rph
        self.add_pph = add_pph
        self.polynomial_degree = polynomial_degree
        self.interaction_terms_only = interaction_terms_only
        
        self.epsilon = epsilon
        self.loss = loss
        self.alpha = alpha
        self.penalty = penalty
        self.l1_ratio = l1_ratio
        
        return
    
    def fit(self, X, y=None):
        self.scaled_epsilon = self.epsilon * y.std()
        self.data_preparer = DataPreparer(self.numerical_features, self.categorical_features, add_bpr=self.add_bpr,
                                          add_rph=self.add_rph, add_pph=self.add_pph, polynomial_degree=self.polynomial_degree,
                                          interaction_terms_only=self.interaction_terms_only)
        self.predictor = SGDRegressor(epsilon=self.scaled_epsilon, loss=self.loss, alpha=self.alpha, penalty=self.penalty,
                                      l1_ratio=self.l1_ratio, max_iter=100_000_000)
        
        self.data_preparer.fit(X)
        prepped_X = self.data_preparer.transform(X)
        self.predictor.fit(prepped_X, y)
        return self
    
    def predict(self, X):
        prepped_X = self.data_preparer.transform(X)
        return self.predictor.predict(prepped_X)

In [25]:
# do a coarse grid/random search with cv=5 to find the best hyperparameter values

# make sure to return train scores so you can compare train and validation scores.
# if there is a large discrepancy between train and validation scores, you are overfitting and should regularize

In [26]:
coarse_polynomial_degree_list = [1, 2, 3]
coarse_alpha_list = [0, 1, 10]
coarse_epsilon_list = [0, 1, 10]
coarse_l1_ratio_list = [.25, .5, .75]

coarse_parameter_grid = [
    {'polynomial_degree' : [2], 'interaction_terms_only' : [True], 'penalty' : ['elasticnet'], 'alpha' : coarse_alpha_list,
     'epsilon' : coarse_epsilon_list, 'l1_ratio' : coarse_l1_ratio_list},
    {'polynomial_degree' : coarse_polynomial_degree_list, 'interaction_terms_only' : [False], 'penalty' : ['elasticnet'], 'alpha' : coarse_alpha_list,
     'epsilon' : coarse_epsilon_list, 'l1_ratio' : coarse_l1_ratio_list},
    {'polynomial_degree' : [2], 'interaction_terms_only' : [True], 'penalty' : ['l2'], 'alpha' : coarse_alpha_list,
     'epsilon' : coarse_epsilon_list},
    {'polynomial_degree' : coarse_polynomial_degree_list, 'interaction_terms_only' : [False], 'penalty' : ['l2'], 'alpha' : coarse_alpha_list,
     'epsilon' : coarse_epsilon_list}
]

In [27]:
coarse_parameter_grid = {'penalty' : ['l2'], 'alpha' : coarse_alpha_list, 'epsilon' : coarse_epsilon_list}

In [28]:
coarse_parameter_grid = {'alpha' : coarse_alpha_list, 'loss' : ['epsilon_insensitive'], 'max_iter' : [100_000_000], 'epsilon' : [10_000]}

In [ ]:
coarse_grid_search = GridSearchCV(SGDRegressor(), coarse_parameter_grid, cv=3, scoring='neg_mean_squared_error', return_train_score=True)
coarse_grid_search.fit(prepped_train_features, train_targets)

In [ ]:
np.sqrt(-coarse_grid_search.cv_results_['mean_test_score'])

In [ ]:
cv_results = coarse_grid_search.cv_results_
for negative_mse, params in sorted(zip(cv_results['mean_test_score'], cv_results['params']), reverse=True, key=lambda zip_pair : zip_pair[0]):
    print(f'{params}  ==  {np.sqrt(-negative_mse)}')

In [ ]:
# do a more refined grid/random search centered on the best hyperparameter values

In [ ]:
# select model hyperparameters and train on full dataset

In [ ]:
# evaluate model on test set